<a href="https://colab.research.google.com/github/alvinicdf-sketch/2026/blob/main/20260108_SplitResume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


In [ ]:
# @title
# 1. 安裝與清理環境

import PyPDF2
import re
import os
from google.colab import files

# 清理先前產生的檔案
!rm -f *.pdf *.zip

# 2. 上傳檔案
print("請上傳您的 PDF 檔案：")
uploaded = files.upload()
if not uploaded:
    print("未上傳檔案。")
else:
    input_pdf_path = list(uploaded.keys())[0]

    def split_resumes_deduplicated(pdf_path):
        reader = PyPDF2.PdfReader(pdf_path)
        total_pages = len(reader.pages)
        resume_segments = []

        # 用於記錄已出現過的姓名
        seen_names = set()

        print(f"\n--- 開始掃描 (共 {total_pages} 頁) ---")

        for page_num in range(total_pages):
            text = reader.pages[page_num].extract_text()

            source_tag = ""
            if "使⽤⼈員: 官昕蕾" in text:
                source_tag = "104"
            elif "聯絡電話" in text:
                source_tag = "1111"

            if source_tag:
                # 防重複機制：避免同一份履歷重複觸發 (間隔需大於 1 頁)
                if resume_segments and (page_num - resume_segments[-1]["start_page"] < 2):
                    continue

                # 姓名抓取邏輯
                name = "姓名識別失敗"
                name_match = re.search(r"([^\s\d\:\：]{2,5})\s*\d+\s*歲", text)

                if name_match:
                    name = name_match.group(1).strip()
                    for noise in ["列印", "管理", "履歷", "頁", "日期", "聯絡"]:
                        if noise in name:
                            name = "未知姓名"
                            break

                # --- 重複姓名過濾邏輯 ---
                if name != "姓名識別失敗" and name != "未知姓名":
                    if name in seen_names:
                        print(f"「重複刪除」：偵測到重複姓名【{name}】於第 {page_num+1} 頁，已自動跳過。")
                        continue
                    else:
                        seen_names.add(name)
                # ----------------------

                resume_segments.append({
                    "start_page": page_num,
                    "name": name,
                    "tag": source_tag
                })
                print(f"[{len(resume_segments)}] 發現 {source_tag}：{name} (第 {page_num+1} 頁)")

        # 3. 執行切割
        found_count = len(resume_segments)
        print(f"\n掃描完成！排除重複後共偵測到 {found_count} 份履歷。開始分割中...")

        output_files = []
        for i in range(found_count):
            start = resume_segments[i]["start_page"]
            end = resume_segments[i+1]["start_page"] if i+1 < found_count else total_pages

            writer = PyPDF2.PdfWriter()
            for p in range(start, end):
                writer.add_page(reader.pages[p])

            clean_name = resume_segments[i]['name'].replace("/", "_")
            tag = resume_segments[i]['tag']
            filename = f"{i+1:02d}_{clean_name}_履歷_{tag}.pdf"

            with open(filename, "wb") as f:
                writer.write(f)
            output_files.append(filename)

        # 4. 打包下載
        if output_files:
            zip_name = 'resumes_deduplicated.zip'
            !zip -q {zip_name} *.pdf
            print(f"\n--- 成功產出 {len(output_files)} 個檔案，準備下載 ---")
            files.download(zip_name)

    split_resumes_deduplicated(input_pdf_path)
    print("已成功下載")

請上傳您的 PDF 檔案：
